<font size=7>Spikesort Interface Source Code

Welcome to the <font color='blue'>Spikesort Interface Source Code</font>! After uploading your data to this interface, choose a <font color='blue'>sorter</font> (Mountainsort, Spyking Circus)to do the spike sorting. After the <font color='blue'>spike sorting</font> is complete, the data can be analyzed from different graphs (e.g. Waveform, Spike Detection) on the interface. After every few times you run a cell on the interface,  remember to press the <font color='red'>clean button</font> and remove excess containers to clean the computer.



# Command to Suppress Warning Messages


Run this code to hide <font color='red'>warning messages</font>.

In [ ]:
import warnings
warnings.filterwarnings("ignore")

# Import Packages

The <font color='blue'> necessary packages </font>to run the <font color='blue'>main source code</font> file are imported here.

In [ ]:
# import packages for spikesorting and visualizing
import kachery as ka
import matplotlib.pyplot as plt
import os
import subprocess
import sys
import numpy as np
def install(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])
import hither_sf as hither
from spikeforest2 import sorters
import matplotlib
import braingeneers.datasets_electrophysiology
import spikeinterface.extractors as se #for mda conversion only
import spiketoolkit as st
import braingeneers
import json
import spikeforest2_utils
from spikeforest2_utils import AutoRecordingExtractor, MdaRecordingExtractor
import hither_sf as hither
import io
import sys
spikeforest2_utils.__version__=1.0
import ipywidgets as ipw
import random
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from braingeneers import datasets_electrophysiology as ephys
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.core.display import HTML, display, Javascript, clear_output
import pickle
import spikeinterface.widgets as sw
from ipywidgets import VBox
spikeforest2_utils.__version__=1.0

# Directories

The <font color='blue'>directories</font> needed for the main source code file are created here.

In [ ]:
# Create directories for saving sorting output
folders = !ls
# Create directory for storing a sorting output if the directory does not exist
if "kach_dir_pi" not in folders:
    !mkdir kach_dir_pi
    !touch kach_dir_pi/raw.mda
current_dir = ! pwd
# Create path to kach_dir to store sorting output
kach_dir = current_dir[0] + "/"+"kach_dir_pi"+"/"


# Objects

The <font color='blue'>objects</font> needed for the <font color='blue'>main source code</font> file are created here.

In [ ]:
# Set the environmental variable that the spike interface uses
fname = os.environ['KACHERY_STORAGE_DIR']=kach_dir
fname


In [ ]:
# Create objects to save future output
SpikeSorter = type('SpikeSorter', (object,), {})()

# Import Data

Run this code to <font color='blue'>import the data</font>  onto the interface.

In [ ]:
# Command given to load the data onto the interface and store in the variables: Spike.X, Spike.t and Spike.fs
#%run 'GetData'.ipynb
#Spike = type('test', (object,), {})()
#Spike.X, Spike.t, Spike.fs = ephys.load_blocks("2020-11-05-e-UCSF-axionplate", 0, start=0, stop=None)

# Spikesorting

The  buttons for the <font color='blue'>sorters</font>, <font color='magenta'>Mountainsort</font> and <font color='magenta'>Spyking Circus</font>, are created. 

<font color='blue'>Click</font> one of the <font color='blue'>buttons</font> to run the <font color='blue'>sorter</font> on the data collected.

The <font color='blue'>spike sorting</font> will identify <font color='blue'>clusters</font> in the <font color='blue'>waveforms</font> and judge whether they came from an individual <font color='blue'>neuron</font>

In [ ]:
# Creates the buttons for the MountainSort and Spyking Circus spikesorters
SpikeSorter.spike_select_btn  = ipw.ToggleButtons(options=['MountainSort4', 'SpykingCircus'] )

In [ ]:
# Creates the button to run the spikesort algorithms depending on the button selected
SpikeSorter.run_btn  = ipw.Button( description="Run",button_style='info', )
#SpikeSorter.run_btn 

In [ ]:
# helper function
def register_recording(*, fname,geom,recording, output_fname, label):
            # Create kachery universal file pointer to raw data recording
            raw_path = ka.store_file(fname  + 'raw.mda')
            # Put raw data parameters and geometry into single recording object
            obj = dict(
                raw=raw_path,
                params=ka.load_object(fname + 'params.json'),
                geom=geom.tolist()
            )
            # Object field is created which stores the object recording
            obj['self_reference'] = ka.store_object(obj, basename='{}.json'.format(label))
            # Write recording object to file
            with open(output_fname, 'w') as f: json.dump(obj, f, indent=4)
            # Returns recording object
            return obj['self_reference']

In [ ]:
# Main function:
def Spikesort(b):
    
    global recording
    global recording_path

    
    # Make dummie geometry for each channel at coordinate (0,0) TEMPORARY
    # The geometry is the the 2-D coordinates of the recording channels
    geom = np.zeros(((Spike.X.shape[1]),2))    # Maybe should be: np.zeros(((Spike.X.shape[0]),2))
    # Transpose recording because spike extractor takes: number of channels * number of timepoints
    X= Spike.X.T
    # When each sample is taken, the time is obtained when the sample is recorded, and added to the list t.
    # Each element of list t: sample # * (1/sampling frequency).
    # EXAMPLE: When the sampling rate is 100 Hz, a recording is supposed to have a sample every 1/100th of a second. 
    # Therefore, the 120th sample of the recording is expected to occur after 1.2 seconds.
    t = Spike.t
    # Define fs as sampling frequency
    fs = Spike.fs
    
    # Load recording from X time series and the geometry and the sampling frequency
    recording=se.NumpyRecordingExtractor( timeseries=X, geom=geom, sampling_frequency=fs )
    # Get path to recording object(a dictionary of the recording data, geometry, sampling frequency, and/or other parameters)
    recording_path =  register_recording( recording=recording, geom=geom, fname=fname, output_fname=fname+'new_recording.json', label='new_recording')
    
    # If the MountainSort button is clicked, run the MountainSort algorithm
    if SpikeSorter.spike_select_btn.value == 'MountainSort4':
        with ka.config(fr='default_readonly'):
            #\th hither.config(cache='default_readwrite'):
            # Use hither to open default container where sorting can be ran
                # Give sorter recording object, destination file, and specify detect threshold
                # Detect threshold: number of standard deviations away from rms noise level to detect spikes
                with hither.config(container='default'):
                    result = sorters.mountainsort4.run(
                        recording_path=recording_path,
                        sorting_out=hither.File(),
                        detect_threshold=3
                        )
                    print("recording out path: ", recording_path)
                    print("sorting out path: ",result.outputs.sorting_out)
                    SpikeSorter.sorting_path = str(result.outputs.sorting_out)
        print(result.outputs.sorting_out)
    # If the Spyking Circus button is clicked, run the Spyking Circus algorithm
    elif SpikeSorter.spike_select_btn.value == 'SpykingCircus':
        with ka.config(fr='default_readonly'):
            #with hither.config(cache='default_readwrite'):
            # Give sorter recording object, destination file, and specify detect threshold
            # Detect threshold: number of standard deviations away from rms noise level to detect spikes
                with hither.config(container='default'):
                    result = sorters.spykingcircus.run(
                            recording_path=recording_path,
                            sorting_out=hither.File(),
                            detect_threshold=3
                            )
                    print("recording out path: ", recording_path)
                    print("sorting out path: ",result.outputs.sorting_out)
                    SpikeSorter.sorting_path = str(result.outputs.sorting_out)
        print(result.outputs.sorting_out)
    # If neither button is selected, request that button is clicked
    else:
        print ('Please select one of the sorting buttons')
    
    print("finished")


In [ ]:
#Assign function to button
SpikeSorter.run_btn.on_click(Spikesort)

In [ ]:
# A widget box displays the MoutainSort4 and SpykingCircus buttons
# Depending on the button selected, a spikesorting algorithm will run
SpikeSorter.Spike_Widget_Box = VBox([VBox([SpikeSorter.spike_select_btn,SpikeSorter.run_btn ])])
SpikeSorter.Spike_Widget_Box

# Graphs

<font color='red'> THIS STEP CAN BE DONE AFTER THE SORTING </font>

The <font color="blue">buttons</font> for the graphs are created. 

<font color='blue'> Run both cells </font> and then <font color='blue'> select a button </font> for the type of graph you would like to see:

<font color='magenta'> w_ts </font> = timeseries of the channels over the recording time

<font color='magenta'> w_sp </font> = spectral power density of the channels

<font color='magenta'> w_rs </font> = spike rasters (firing over time of each putative neuron)

<font color='magenta'> w_isi </font> = distribution of each neuron's firing rate

<font color='magenta'> w_ach </font> = one neuron's firing rate compared to itself (autocorrelation chart)

<font color='magenta'> w_cch </font> = shows correlation between neurons

<font color='magenta'> w_wf </font> = average waveform of each neuron's action potential

<font color='magenta'> w_ampd </font> = amplitude distribution of each neuron's action potential

<font color='magenta'> w_ampt </font> = amplitude time series shows amplitude of a neuron's firing over time

 If you want to see the <font color="magenta">time series graph</font>, click one of the other <font color="blue">buttons</font>, then hit the time series button.

<b><font color="red">To do</font></b>
* Create function that saves output if the experiment was run previously to load the sorting automatically
  Link for github: https://github.com/SpikeInterface/spikeextractors/blob/master/spikeextractors/extractors/mdaextractors/mdaextractors.py

In [ ]:
# Creates buttons to display the graphs: w_ts, w_sp, w_rs, w_isi, w_ach, w_cch, w_wf, w_ampd, w_ampt
# waveforms
Graph_Button = ipw.ToggleButtons(options=['time series','spectral power','spike rasters','interspike intervals','autocorrelation','cross-correlation', 'amplitude distribution','amplitude time series'] ) 
Graph_Button

In [ ]:
from spikeforest2_utils import AutoRecordingExtractor, AutoSortingExtractor
from IPython.display import display, clear_output

def Graphs(change):
    
    clear_output()
    display(Graph_Button)
    
    # Arpitha Describe what this does
    global sorting_true
    # The sorting path is too long and has extraneous characters
    # Trim extraneous characters to load local sorting
    sorting_out_short = SpikeSorter.sorting_path[12:-1]
    # Get path(universal kachery path) to stored sorting object
    universal_sorting_path = ka.store_file(sorting_out_short)
    # Load recording object from recording path
    recording = AutoRecordingExtractor(recording_path, download=False)
    # Load sorting object from sorting path
    sorting_true = AutoSortingExtractor(universal_sorting_path)
    
    # Make graph based on user selection
    if Graph_Button.value == 'time series':
        print('Timeseries of the channels over the recording time')
        w_ts_btn = sw.plot_timeseries(recording)
        plt.show()
   
    elif Graph_Button.value == 'spectral power':
        print('Spectral power density of the channels')
        w_sp_btn = sw.plot_spectrum(recording)
        plt.show()
    
    elif Graph_Button.value == 'spike rasters':
        print('Spike rasters : firing over time of each putative neuron')
        w_rs_btn = sw.plot_rasters(sorting_true, sampling_frequency=Spike.fs)
        plt.show()
        
    elif Graph_Button.value == 'interspike intervals':
        print('Distribution of each neurons firing rate')
        w_isi_btn = sw.plot_isi_distribution(sorting_true, sampling_frequency=Spike.fs, bins=10, window=1)
        plt.show()
        
    elif Graph_Button.value == 'autocorrelation':
        print('One neurons firing rate compared to itself (autocorrelation chart)')
        w_ach_btn = sw.plot_autocorrelograms(sorting_true, sampling_frequency=Spike.fs, bin_size=1, window=10, unit_ids=[1, 2, 4, 5, 8, 10, 7])
        plt.show()
        
    elif Graph_Button.value == 'cross-correlation':
        print('Shows correlation between neurons')
        w_cch_btn = sw.plot_crosscorrelograms(sorting_true, sampling_frequency=Spike.fs, unit_ids=[1, 5, 8], bin_size=0.1, window=5)
        plt.show()
        
    elif Graph_Button.value == 'waveforms':
        print('Average waveform of each neuron action potential')
        w_wf_btn = sw.plot_unit_waveforms(recording, sorting_true,unit_ids=sorting_true.get_unit_ids(),  max_spikes_per_unit=100)
        plt.show()
        
    elif Graph_Button.value == 'amplitude distribution':
        print('Amplitude distribution of each neurons action potential')
        w_ampd_btn = sw.plot_amplitudes_distribution(recording, sorting_true, max_spikes_per_unit=300)
        plt.show() 
    
    elif Graph_Button.value == 'amplitude time series':
        print('Amplitude time series shows amplitude of a neurons firing over time')
        w_ampt_btn = sw.plot_amplitudes_timeseries(recording, sorting_true, max_spikes_per_unit=300)
        plt.show()
        
    else:
        print ('Please select one of the sorting buttons')
        
Graph_Button.observe( Graphs , names = 'value') 


# Waveforms

<font color='red'> THIS STEP CAN BE DONE AFTER THE SORTING </font>

The <font color='magenta'>Waveform graphs</font> are the <font color='blue'>average waveforms</font> of a <font color='blue'>putative action potential</font>. Each graph represents one <font color='blue'>putative neuron</font>, and the <font color='blue'>average signal</font> from each time the neuron fired. Each horizontal line represents the <font color='blue'>voltage</font> measured at a <font color='blue'>distinct channel</font>.

<font color='blue'>Run the 2 cells</font> below and <font color='blue'>select the button</font> to get a slider. Move the <font color='blue'>slider</font> to select a unit of data to analyze the <font color='blue'>waveform</font> of. Then, click the <font color='blue'>plot button</font> to generate a <font color='magenta'>waveform graph</font> for a particular unit of data.

In [ ]:
# create a start button to generate the slider
wave_btn_start= ipw.Button(description="Start")
wave_btn_start

In [ ]:
def start_wave(b):
    # clears the output from the previous result
    clear_output()
    wave_ids= sorting_true.get_unit_ids()
    # creates a slider that will be displayed once the start button is clicked
    wave_slider= ipw.IntSlider( value=wave_ids[0], min=wave_ids[0], max=wave_ids[-1] )
    display(wave_slider)
    
    # create a button to plot the waveform
    wave_btn_plot = ipw.Button( description="Plot" ) 
    # display the button
    display(wave_btn_plot)
    def wave_plot_func(b):
        # plot the waveform
        sw.plot_unit_waveforms(recording, sorting_true,unit_ids=[wave_slider.value],  max_spikes_per_unit=100)
    wave_btn_plot.on_click(wave_plot_func) 
    
# enable the start_wave button to be clicked
wave_btn_start.on_click(start_wave)

# Spike Detection Graphs

<font color='red'> THIS STEP CAN BE DONE AFTER THE SORTING </font>

<font color='magenta'>Spike detection graphs</font> are the <font color='blue'>waveforms</font> for each channel plotted to look for spikes across all the <font color='blue'>channels</font>

<font color='blue'>Run the 2 cells</font> below and <font color='blue'>select the button</font> to see the <font color='magenta'>spike detection graphs</font> of the data collected.

In [ ]:
# Create a Spike Detection button
Spike_Detection_Button = ipw.Button(description="Spike Detection",button_style='info',)
Spike_Detection_Button

In [ ]:
# Make comments on this block of code with Nico
from spiketoolkit import postprocessing as pp

def SpikeDetection(b):
    
    wav = pp.get_unit_waveforms(recording, sorting_true,unit_ids=sorting_true.get_unit_ids(), ms_before=1, ms_after=2, recompute_info=True, save_property_or_features=False)
    
    units=wav[0].shape[1] #0 indexed
    n_channels=1 #0 indexed?
    n_timepoints=wav[0].shape[2] #all

    try: # Not as many units as other channels since it may cause unnecessary 
         # errors. Avoiding error messages with try and except.
        for this_unit in sorting_true.get_unit_ids():
            for this_chan in range(0,n_channels): 
                average = wav[this_unit][0][this_chan] * 0
                for thing in range(0,n_timepoints): #plot all occurences of unit n_spikes
                    fig = plt.plot(wav[this_unit][thing][this_chan],'gray')
                    average = average + wav[this_unit][thing][this_chan]
                    the_title="Spike Detection on Unit ,"+str(this_unit)+", channel "+str(this_chan)
                    plt.xlabel('3 milliseconds total')
                    plt.ylabel('microvolts')
                    plt.title(the_title)
                plt.plot(average/n_timepoints)
                plt.show()
    except:
        pass

# Assign function to button
Spike_Detection_Button.on_click(SpikeDetection)


# Remove Containers

<font color="red">THIS IS A REALLY IMPORTANT STEP</font>

The <font color="blue"> clean button</font> has been created.

To do <font color="blue">spikesorting</font> we use docker. <font color="blue">Docker</font> takes up considerable <font color="blue">computer memory</font>  because it creates <font color="blue">containers</font> to run the <font color="magenta">spikesorters</font>.

<font color='blue'>Run the 2 cells</font> and <font color='blue'>select the button</font> to <font color='blue'>remove containers</font> and clean the computer.

In [ ]:
# Creates a button to remove containers
Clean_Button = ipw.Button( description="CLEAN",button_style='danger', )
Clean_Button

In [ ]:
def Clean(b):
    # Code to clean the containers
    !sudo docker ps -a | grep Exit | cut -d ' ' -f 1 | xargs sudo docker rm
    print("Computer has been cleaned")
# Assign function to button
Clean_Button.on_click(Clean)